**1. Импортирование необходимых библиотек.**

In [1]:
import pandas as pd
import numpy as np
from feature_engine.creation import MathFeatures
from sklearn.model_selection import train_test_split
import os

**2. Предобработка данных.**

**2.1. Загрузка данных.**

In [2]:
cwd = os.getcwd()
filepath="\\data.csv"
data = pd.read_csv(filepath, delimiter=";")

FileNotFoundError: [Errno 2] No such file or directory: '\\data.csv'

**2.2. Просмотр данных.**

In [13]:
data.head()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
0,1,1,95,67,72,20,0,0
1,0,1,64,67,64,0,0,0
2,0,1,58,69,56,0,0,0
3,0,1,39,58,68,0,0,0
4,0,1,46,52,53,0,0,0


In [14]:
data.describe()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
count,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000,116.000000
mean,0.206897,1.258621,69.732759,71.017241,68.620690,4.948276,0.025862,0.017241
std,0.406838,0.512805,15.240311,13.480409,12.522436,6.220366,0.159412,0.130734
min,0.000000,1.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,64.000000,64.000000,62.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,74.000000,72.000000,71.500000,3.000000,0.000000,0.000000
75%,0.000000,1.000000,80.000000,80.250000,78.000000,10.000000,0.000000,0.000000
max,1.000000,3.000000,95.000000,100.000000,94.000000,30.000000,1.000000,1.000000


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype
---  ------                             --------------  -----
 0   Пол                                116 non-null    int64
 1   Приоритет                          116 non-null    int64
 2   Балл по математике                 116 non-null    int64
 3   Балл по русскому языку             116 non-null    int64
 4   Балл по выбранному предмету        116 non-null    int64
 5   Балл за индивидуальные достижения  116 non-null    int64
 6   Наличие серебрянной медали         116 non-null    int64
 7   Наличие золотой медали             116 non-null    int64
dtypes: int64(8)
memory usage: 7.4 KB


In [16]:
data.isnull().sum()

Пол                                  0
Приоритет                            0
Балл по математике                   0
Балл по русскому языку               0
Балл по выбранному предмету          0
Балл за индивидуальные достижения    0
Наличие серебрянной медали           0
Наличие золотой медали               0
dtype: int64

**2.3. Генерация новых признаков.**

In [17]:
mf = MathFeatures(variables=["Балл по математике", "Балл по русскому языку", "Балл по выбранному предмету"], func = "mean", new_variables_names=["Средний балл"])
mf.fit(data)
mf.transform(data)

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали,Средний балл
0,1,1,95,67,72,20,0,0,78.000000
1,0,1,64,67,64,0,0,0,65.000000
2,0,1,58,69,56,0,0,0,61.000000
3,0,1,39,58,68,0,0,0,55.000000
4,0,1,46,52,53,0,0,0,50.333333
...,...,...,...,...,...,...,...,...,...
111,0,1,72,75,80,0,0,0,75.666667
112,0,1,70,61,62,0,0,0,64.333333
113,1,1,70,60,53,4,0,0,61.000000
114,0,1,52,46,46,0,0,0,48.000000


**3. Кластеризация.**

**3.1. Класс сети Кохонена.**

In [40]:
class KohonenNet():
    def __init__(self, m=3, n=3, lr=1, sigma=1, max_iter=3000, weights = []):
        self.m = m #количество классов
        self.n = n #количество уровней сети
        self.shape = (m, n)
        self.initial_lr = lr
        self.lr = lr #скорость обучения
        self.sigma = sigma #параметр изменения скорости обучения (она будет снижаться с каждой итерацией)
        self.max_iter = max_iter
        self.weights = np.stack([np.float64(np.array(weights))]*self.m, axis=0)

#функция нахождения выйгравшего нейрона (нейрона с минимальным расстоянием до точки)
    def find_bmu(self, x):
        x_stack = np.stack([x]*(self.m), axis=0)#матрица размером [количество центров кластеров(нейронов), количество признаков] (для каждого веса своя строка признаков(точка))
        distance = np.linalg.norm(x_stack - self.weights, axis=1)#эвклидовы расстояния между точкой выборки и нейронами
        return np.argmin(distance) #возвращаем индекс минимального расстояния (индекс подходящего нейрона)

    def step(self, x):
        bmu_index = self.find_bmu(x)#передаем в функцию строку признаков (точку) и получаем индекс выйгравшего нейрона (центра кластера)
        self.weights[bmu_index] += self.lr * (x - self.weights[bmu_index])

    #обучение весов
    def fit(self, X, epochs=1, shuffle=True):
        global_iter_counter = 0
        n_samples = X.shape[0]#количество элементов выборки
        total_iterations = np.minimum(epochs * n_samples, self.max_iter)#количество обучения весов в эпохе

        for epoch in range(epochs):#изначально у нас 1 эпоха
            if global_iter_counter > self.max_iter:#не даем проводить больше 3000 эпох
                break

            if shuffle:
                indices = np.random.permutation(n_samples)#создание массива с индексами в разброс
            else:
                indices = np.arange(n_samples)#создание массива с индексами по порядку

            #обучение
            for idx in indices:#проход по выборке с индексами idx
                if global_iter_counter > self.max_iter:
                    break
                input = X[idx]
                #перемещение нейрона(изменение весов победившего нейрона)
                self.step(input)

                global_iter_counter += 1
                self.lr = (1 - (global_iter_counter / total_iterations)) * self.initial_lr#изменение параметра обучения

        self._n_iter_ = global_iter_counter

        return

    def return_changed_weights(self):
      return self.weights

    def predict(self, X):
        labels = np.array([self.m - 1 - self.find_bmu(x) for x in X])
        return labels

**3.2. Подбор весов.**

In [179]:
for i in range(5):
    weights = np.random.uniform(low=-1, high=1, size=data.shape[1])
    print(f"Стартовые веса:\n{weights}")
    kohonen = KohonenNet(m=4, n=1, weights=weights)
    kohonen.fit(np.array(data))
    preds = pd.DataFrame(kohonen.predict(np.array(data)), columns=["Успеваемость за 1 семестр"])
    print(f"Результат кластеризации:\n{preds.value_counts()}")

Стартовые веса:
[ 0.8160717   0.9195155   0.7762923   0.2676749   0.19947087  0.78323143
 -0.23505747 -0.63120857]
Результат кластеризации:
Успеваемость за 1 семестр
3                            115
2                              1
Name: count, dtype: int64
Стартовые веса:
[ 0.85054541 -0.28280555  0.97931434  0.63944002  0.13388873 -0.62773075
  0.60529966  0.11458828]
Результат кластеризации:
Успеваемость за 1 семестр
3                            113
1                              2
2                              1
Name: count, dtype: int64
Стартовые веса:
[-0.62214809  0.27958174  0.17396178  0.01283714  0.4429734  -0.77221009
 -0.94108155 -0.61523694]
Результат кластеризации:
Успеваемость за 1 семестр
3                            89
1                            26
2                             1
Name: count, dtype: int64
Стартовые веса:
[ 0.14083163 -0.46849622 -0.74270047 -0.79240536  0.45124787  0.80243658
 -0.17944016  0.12687049]
Результат кластеризации:
Успеваемость за 1 семес

**3.3. Кластеризация с объектов выборки с помощью сети Кохонена.**

In [198]:
weights = [-0.96207473, 0.78578409, 0.21926883, 0.92755785, 0.99987121, -0.76948015, -0.70926396, -0.86169305]
print(f"Стартовые веса:\n{weights}")
kohonen = KohonenNet(m=4, n=1, weights=weights)
kohonen.fit(np.array(data))
preds = pd.DataFrame(kohonen.predict(np.array(data)), columns=["Успеваемость за 1 семестр"])
print(f"Результат кластеризации:\n{preds.value_counts()}")

Стартовые веса:
[-0.96207473, 0.78578409, 0.21926883, 0.92755785, 0.99987121, -0.76948015, -0.70926396, -0.86169305]
Результат кластеризации:
Успеваемость за 1 семестр
3                            115
2                              1
Name: count, dtype: int64


In [184]:
kohonen.weights

array([[ 2.38486025e-01,  1.31281644e+00,  7.63343993e+01,
         7.67878455e+01,  7.20266262e+01,  5.23323704e+00,
         1.34850003e-01,  2.32604892e-02],
       [ 5.16137161e-03,  1.00000000e+00,  4.51613290e+01,
         5.78924523e+01,  5.06315088e+01,  2.27011515e-02,
        -4.01109564e-10, -4.87312684e-10],
       [-6.46912318e-01,  8.55958267e-01,  1.47439386e-01,
         6.23702692e-01,  1.63964651e+01, -5.17409066e-01,
        -4.76918870e-01, -5.79414292e-01],
       [-9.62074730e-01,  7.85784090e-01,  2.19268830e-01,
         9.27557850e-01,  9.99871210e-01, -7.69480150e-01,
        -7.09263960e-01, -8.61693050e-01]])

Согласно предсказаниям сети Кохонена выборка обладает дисбалансом классов

**4. Разделение данных.**

In [13]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

**5. Генерация новых данных.**

In [14]:
from tabgan.sampler import OriginalGenerator, GANGenerator, ForestDiffusionGenerator, LLMGenerator

C:\Users\vecni\anaconda3\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [15]:
new_X_train, new_y_train = OriginalGenerator().generate_data_pipe(data, target, data)
new_X_train, new_y_train

(     Пол  Приоритет  Балл по математике  Балл по русскому языку  \
 0      0          1                  46                      67   
 1      0          1                  46                      67   
 2      0          1                  46                      67   
 3      0          1                  46                      67   
 4      0          1                  52                      51   
 ..   ...        ...                 ...                     ...   
 326    0          1                  78                      83   
 327    0          1                  88                      83   
 328    0          1                  88                      83   
 329    0          1                  88                      83   
 330    0          1                  90                      83   
 
      Балл по выбранному предмету  Балл за индивидуальные достижения  \
 0                             64                                  0   
 1                             64     

In [16]:
pd.DataFrame(new_y_train).describe()
pd.DataFrame(new_X_train).describe()

,Пол,Приоритет,Балл по математике,Балл по русскому языку,Балл по выбранному предмету,Балл за индивидуальные достижения,Наличие серебрянной медали,Наличие золотой медали
count,331.000000,331.000000,331.000000,331.000000,331.000000,331.000000,331.000000,331.000000
mean,0.145015,1.241692,68.864048,70.673716,68.287009,4.383686,0.039275,0.012085
std,0.352649,0.481993,12.994740,11.068487,11.215484,5.572191,0.194542,0.109429
min,0.000000,1.000000,39.000000,42.000000,39.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,64.000000,64.000000,62.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,72.000000,72.000000,70.000000,1.000000,0.000000,0.000000
75%,0.000000,1.000000,78.000000,78.000000,77.000000,10.000000,0.000000,0.000000
max,1.000000,3.000000,90.000000,97.000000,92.000000,25.000000,1.000000,1.000000


In [17]:
pd.DataFrame(new_y_train).describe()

,Успеваемость за 1 семестр
count,331.0
mean,2.0
std,0.0
min,2.0
25%,2.0
50%,2.0
75%,2.0
max,2.0
